## Imports and Data

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

england_matches = pd.read_json('new_data/matches/matches_England.json')
teams = pd.read_json('new_data/teams.json')
england_events = pd.read_json('new_data/events/events_England.json')
players = pd.read_json('new_data/players.json')

## Team Analysis
### Team class for accessing team attributes (id, matches, players, events etc.) 


In [2]:
class team:
    def __init__(self, name:str):
        self.name = name
        self.team_id = self.get_team_id()
        self.matches = self.get_matches()
        self.players = self.get_players()
        self.all_events = self.get_all_events()
    
    def get_team_id(self):
        return int(teams[teams['name'] == self.name]['wyId'])
        
    def get_matches(self, opposition_team=None):
        matches = pd.DataFrame(columns=england_matches.columns)
        specific_matches = pd.DataFrame(columns=england_matches.columns)
        
        for index,row in england_matches.iterrows():
            teams_in_match = [int(team) for team in list(row['teamsData'].keys())]
            if opposition_team is None:
                if self.team_id in teams_in_match:
                    matches = matches.append(row)
            else:
                if self.team_id in teams_in_match and opposition_team.team_id in teams_in_match:
                    specific_matches = specific_matches.append(row)
                
        if opposition_team is None:     
            return matches
        else:
            return specific_matches
    
    def get_players(self):
        return players[players['currentTeamId'] == self.team_id]
    
    def get_all_events(self):
        return england_events[england_events['teamId'] == self.team_id]

## Graphics Generation

In [3]:
from matplotlib.patches import Arc

def draw_pitch():
    fig = plt.figure()
    fig.set_size_inches(14,7)

    ax = plt.subplot(111)
    
    #touchline
    plt.plot([0,0], [-5,105], 'k') #left length
    plt.plot([0,100], [105,105], 'k') #upper width
    plt.plot([100,100], [-5,105], 'k') #right length
    plt.plot([100,0], [-5,-5], 'k') #lower width
    plt.plot([50,50], [-5,105], 'k') #halfway line

    #center
    center_circle = plt.Circle((50,50), radius=7, color='black', fill=False)
    center_spot = plt.Circle((50,50), radius=0.7, color='black', fill=True)
    ax.add_artist(center_circle)
    ax.add_artist(center_spot)

    #left 18 yard box
    plt.plot([0,12], [32,32], 'k')
    plt.plot([0,12], [69,69], 'k')
    plt.plot([12,12], [32,69], 'k')
    left_d = Arc((6, 50), width=18.3, height=20, theta1=310, theta2=50, color='black')
    ax.add_patch(left_d)

    #left 6 yard box
    plt.plot([0,4], [41.25,41.25], 'k')
    plt.plot([0,4], [59.75,59.75], 'k')
    plt.plot([4,4], [59.75,41.25], 'k')

    #right 18 yard box
    plt.plot([100,88], [32,32], 'k')
    plt.plot([100,88], [69,69], 'k')
    plt.plot([88,88], [32,69], 'k')
    right_d = Arc((94, 50), width=18.3, height=20, theta1=130, theta2=230, color='black')
    ax.add_patch(right_d)

    #right 6 yard box
    plt.plot([100,96], [41.25,41.25], 'k')
    plt.plot([100,96], [59.75,59.75], 'k')
    plt.plot([96,96], [59.75,41.25], 'k')

#dataframe should be an 'events' dataframe
def draw_heatmap(dataframe):
    sns.kdeplot(dataframe['x_start'], dataframe['y_start'], shade=True, shade_lowest=False, cmap='Greens', n_levels = 30)
    plt.xlim, plt.ylim = (0,100), (0,100)

    plt.axis('off')
    plt.show()

## Dashboard

In [52]:
from ipywidgets import interact, interactive, interact_manual, widgets
import warnings 
warnings.filterwarnings('ignore')

def dashboard():
    #global variables whose values will be updated with every successive selection : filtering achieved through modification of these global vars
    team_1, team_2 = None, None
    matches = None
    target_match = None
    target_team = None
    events = None
    selected_events = None

    #First selector that user sees is team selector : get its options
    england_teams = pd.DataFrame(columns=teams.columns)
    for index,row in teams.iterrows():
        if row['area']['name'] == 'England':
            england_teams = england_teams.append(row)
    team_selector_options = sorted(england_teams['name'].tolist())

    #create team selectors - first interactive element to be called, invocation is on last line 
    team_1_selector = widgets.Dropdown(options=team_selector_options, description='Team 1:')
    team_2_selector = widgets.Dropdown(options=team_selector_options, description='Team 2:')

    def select_teams(team_1_selector, team_2_selector):
        global team_1, team_2, matches

        team_1 = team(team_1_selector)
        team_2 = team(team_2_selector)
        matches = team_1.get_matches(team_2)

        #teams selected, call match selector to select one of 2 matches between them
        match_selector = widgets.Dropdown(options=matches['label'].tolist(), description='Match:')
        interact(select_match, match_selector=match_selector)

    def select_match(match_selector) -> pd.DataFrame:
        global matches, target_match, team_1, team_2

        target_match = matches[matches['label'] == match_selector]
    #     display(target_match)
    
        #match selected, select which team in match to analyse
        target_team_selector = widgets.Dropdown(options=[team_1.name, team_2.name], description='Target Team:')
        interact(select_team, team_selector=target_team_selector)

    def select_team(team_selector):
        global team_1, team_2, target_match, target_team, events

        target_team = team_1 if team_selector == team_1.name else team_2

        events = target_team.all_events[target_team.all_events['matchId'] == int(target_match['wyId'])] 
        
        #unwrap xy coords of events : originally stored as dictionary elements in column 'positions'
        events['x_start'] = events.apply(lambda row: row['positions'][0]['x'], axis=1)
        events['y_start'] = events.apply(lambda row: row['positions'][0]['y'], axis=1)
        
        #event times measured from start of every half => have to check which period an event is in : avoid by measuring all events from start of first half
        events.loc[events.matchPeriod == '2H', 'eventSec'] = 45*60 + events['eventSec']

        #target team selected, now select type of event to analyse
        event_selector = widgets.Dropdown(options=sorted(set(events['eventName'].tolist())), description='event')
        interact(select_event, event_selector=event_selector)
        
    def select_event(event_selector):
        global events, selected_events
        
        selected_events = events[events['eventName'] == event_selector]
        
        #type of event selected, now select time interval 
        range_selector = widgets.IntRangeSlider(value=[0, 90], min=0, max=90, step=10,description='Interval:',disabled=False)
        interact(select_range, range_selector=range_selector)

    def select_range(range_selector):
        global events, selected_events

        target_events = selected_events[selected_events['eventSec'] >= range_selector[0] * 60]
        target_events = target_events[target_events['eventSec'] <= range_selector[1] * 60]
        
        print(target_events.shape) #for debugging
        
        #everything selected, lezzgo
        draw_pitch()
        #prevent heatmap generation for sparse dataframes
        try:
            draw_heatmap(target_events)
        except:
            print("Not Enough Events For Density Estimation")

    #invoke team selector : get the ball rolling
    interact_manual(select_teams, team_1_selector=team_1_selector, team_2_selector=team_2_selector)

In [53]:
dashboard()

interactive(children=(Dropdown(description='Team 1:', options=('AFC Bournemouth', 'Arsenal', 'Brighton & Hove …